In [13]:
import pandas as pd
import numpy as np

housing = pd.read_csv("../data/interim/housing_data.csv", index_col=0)

housing['Neighborhood_Name'] = housing['Neighborhood_Name'].apply(lambda x: str(x).replace(" ",""))

housing.head()

,OBJECTID,NEIGH_NO,Neighborhood_Name,Neighborhood_Type,Neighborhood_Subtype,ACS_Vinatage,Occupied_Housing_Units,Owner_Households,Renter_Households,Gross_Rent_as_a_Percentage_of_Household_Income_Less_than_15.0%,...,"Owner_Costs_Less_Than_$1,000","Owner_Costs_$1,000-$1,499","Owner_Costs_$1,500-$1,999","Owner_Costs_$2,000-$2,499","Owner_Costs_$2,500-$2,999","Owner_Costs_$3,000-$3,499","Owner_Costs_$3,500-$3,999","Owner_Costs_$4,000_or_more",Owner_housing_units_without_a_mortgage,Neighborhood_Type_(outside_comp_plan_areas_id)
0,1,1.0,CouncilDistrict1,CD,NaN,5Y23,48209,27781,20428,2120,...,225,700,2397,2529,3747,3651,2926,5002,6604,CD
1,2,2.0,CouncilDistrict2,CD,NaN,5Y23,43824,21520,22304,2609,...,212,370,1700,2311,2311,2511,1916,3241,6948,CD
2,3,3.0,CouncilDistrict3,CD,NaN,5Y23,60156,20642,39514,6330,...,335,293,867,1469,1765,1845,1620,6162,6286,CD
3,4,4.0,CouncilDistrict4,CD,NaN,5Y23,42569,18939,23630,2668,...,67,252,721,1165,1544,1936,1205,5887,6162,CD
4,5,5.0,CouncilDistrict5,CD,NaN,5Y23,47887,23856,24031,2544,...,130,501,1419,2661,3083,2561,1726,4846,6929,CD


In [14]:
mortgage = housing[['Neighborhood_Name','Owner_Households','Owner_housing_units_with_a_mortgage_where_selected_monthly_onwer_costs_as_a_percentage_of_household_income_can_be_computed','Selected_Monthly_Owner_Costs_as_a_Percentage_of_Household_Income_Less_than_20.0%','Selected_Monthly_Owner_Costs_as_a_Percentage_of_Household_Income_20.0_to_24.9%','Selected_Monthly_Owner_Costs_as_a_Percentage_of_Household_Income_25.0_to_29.9%','Selected_Monthly_Owner_Costs_as_a_Percentage_of_Household_Income_30.0_to_34.9%','Selected_Monthly_Owner_Costs_as_a_Percentage_of_Household_Income_35.0%_or_more']]

new_cols = ['Neighborhood_Name','Owner_Households','Non-Zero_Cost_Owner_Households','Owner_Costs_lt_20.0%','Owner_Costs_20.0-24.9%', 'Owner_Costs_25.0-29.9%','Owner_Costs_30.0-34.9%','Owner_Costs_Over_35']

mortgage.columns = new_cols

mortgage

,Neighborhood_Name,Owner_Households,Non-Zero_Cost_Owner_Households,Owner_Costs_lt_20.0%,Owner_Costs_20.0-24.9%,Owner_Costs_25.0-29.9%,Owner_Costs_30.0-34.9%,Owner_Costs_Over_35
0,CouncilDistrict1,27781,21177,9655,3853,2598,1247,3778
1,CouncilDistrict2,21520,14572,6440,2246,1493,1171,3188
2,CouncilDistrict3,20642,14356,7393,1949,1305,609,3074
3,CouncilDistrict4,18939,12777,6945,1698,1211,797,2105
4,CouncilDistrict5,23856,16927,7488,2914,1873,1172,3435
...,...,...,...,...,...,...,...,...
87,UniversityDistrict,702,357,83,37,11,17,209
88,Uptown,1772,1326,486,246,79,117,256
89,Downtown,4979,3495,1672,714,334,178,597
90,FirstHill/CapitolHill,5017,3608,1730,681,386,95,705


In [15]:
key = pd.read_csv("../data/processed/Neighborhood_alt_key.csv", index_col=0).set_index("alt_names_list")

mortgage_merge = mortgage.join(key, on='Neighborhood_Name', how='inner', rsuffix='_alt_key')
mortgage_merge = mortgage_merge.groupby("L_HOOD").agg(sum)

/tmp/ipykernel_10505/656535659.py:4: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  mortgage_merge = mortgage_merge.groupby("L_HOOD").agg(sum)


In [16]:
mortgage_merge['Total_Under_25'] = np.sum(mortgage_merge[['Owner_Costs_lt_20.0%','Owner_Costs_20.0-24.9%']],axis= 1)
mortgage_merge['Ratio_Under_25'] = mortgage_merge['Total_Under_25'] / mortgage_merge['Owner_Households']
mortgage_merge.head()

,Neighborhood_Name,Owner_Households,Non-Zero_Cost_Owner_Households,Owner_Costs_lt_20.0%,Owner_Costs_20.0-24.9%,Owner_Costs_25.0-29.9%,Owner_Costs_30.0-34.9%,Owner_Costs_Over_35,S_HOOD_ALT_NAMES,Total_Under_25,Ratio_Under_25
L_HOOD,,,,,,,,,,,
Ballard,WhittierHeightsBallardBallard,9270,7563,3567,1268,762,680,1271,"LoyalHeights,Adams,WhittierHeights,WestWoodlan...",4835,0.521575
Beacon Hill,NorthBeaconHill,1651,1027,354,237,164,40,232,"NorthBeaconHill,Mid-BeaconHill,SouthBeaconHill...",591,0.357965
Capitol Hill,NorthCapitolHillMillerParkMadisonParkMadison-M...,6184,4227,2164,631,510,126,781,"PortageBay,Montlake,MadisonPark,Broadway,Steve...",2795,0.451973
Cascade,EastlakeSouthLakeUnion,1323,853,472,144,67,64,106,"Westlake,Eastlake,SouthLakeUnionWestlake,Eastl...",616,0.465608
Central Area,JudkinsPark,508,271,113,44,48,25,41,"Madrona,Harrison/Denny-Blaine,Minor,Leschi,Man...",157,0.309055


In [17]:
mortgage_merge.to_csv("../data/processed/mortgage_ratio.csv")
mortgage_merge

,Neighborhood_Name,Owner_Households,Non-Zero_Cost_Owner_Households,Owner_Costs_lt_20.0%,Owner_Costs_20.0-24.9%,Owner_Costs_25.0-29.9%,Owner_Costs_30.0-34.9%,Owner_Costs_Over_35,S_HOOD_ALT_NAMES,Total_Under_25,Ratio_Under_25
L_HOOD,,,,,,,,,,,
Ballard,WhittierHeightsBallardBallard,9270,7563,3567,1268,762,680,1271,"LoyalHeights,Adams,WhittierHeights,WestWoodlan...",4835,0.521575
Beacon Hill,NorthBeaconHill,1651,1027,354,237,164,40,232,"NorthBeaconHill,Mid-BeaconHill,SouthBeaconHill...",591,0.357965
Capitol Hill,NorthCapitolHillMillerParkMadisonParkMadison-M...,6184,4227,2164,631,510,126,781,"PortageBay,Montlake,MadisonPark,Broadway,Steve...",2795,0.451973
Cascade,EastlakeSouthLakeUnion,1323,853,472,144,67,64,106,"Westlake,Eastlake,SouthLakeUnionWestlake,Eastl...",616,0.465608
Central Area,JudkinsPark,508,271,113,44,48,25,41,"Madrona,Harrison/Denny-Blaine,Minor,Leschi,Man...",157,0.309055
Delridge,NorthDelridgeHighPointRiverviewHighlandParkHig...,9088,7053,3418,1132,823,508,1163,"HighlandPark,NorthDelridge,Riverview,HighPoint...",4550,0.500660
Downtown,FirstHillBelltown,3728,2530,1347,541,301,105,236,"Pike-Market,Belltown,InternationalDistrict,Cen...",1888,0.506438
Greater Duwamish,SouthParkGeorgetownSouthPark,1795,1392,642,267,208,72,203,"IndustrialDistrict,Georgetown,SouthPark,Harbor...",909,0.506407
Interbay,Interbay,2116,1521,632,247,260,146,197,Interbay,879,0.415406


In [18]:
import geopandas as gpd

map_data = gpd.read_file('../data/raw/Neighborhood_Map_Atlas_Districts.geojson')
map_data.head()

ERROR 1: PROJ: proj_create_from_database: Open of /home/colin/anaconda3/envs/capstone/share/proj failed


,OBJECTID,L_HOOD,S_HOOD_ALT_NAMES,Shape__Area,Shape__Length,geometry
0,28,Ballard,"Loyal Heights, Adams, Whittier Heights, West W...",1.046035e+08,59248.568772,"POLYGON ((-122.40266 47.69602, -122.40236 47.6..."
1,29,Beacon Hill,"North Beacon Hill, Mid-Beacon Hill, South Beac...",1.479894e+08,78783.502732,"POLYGON ((-122.31281 47.59581, -122.31216 47.5..."
2,30,Capitol Hill,"Portage Bay, Montlake, Madison Park, Broadway,...",1.068562e+08,75147.170565,"MULTIPOLYGON (((-122.29599 47.64184, -122.2960..."
3,31,Cascade,"Westlake, Eastlake, South Lake Union",2.717920e+07,48680.288698,"POLYGON ((-122.32124 47.65231, -122.32113 47.6..."
4,32,Central Area,"Madrona, Harrison/Denny-Blaine, Minor, Leschi,...",9.527258e+07,52688.475894,"POLYGON ((-122.27911 47.62786, -122.27915 47.6..."


In [20]:
test = map_data.set_index('L_HOOD').join(mortgage_merge, rsuffix='_hosuing')


,OBJECTID,S_HOOD_ALT_NAMES,Shape__Area,Shape__Length,geometry,Neighborhood_Name,Owner_Households,Non-Zero_Cost_Owner_Households,Owner_Costs_lt_20.0%,Owner_Costs_20.0-24.9%,Owner_Costs_25.0-29.9%,Owner_Costs_30.0-34.9%,Owner_Costs_Over_35,S_HOOD_ALT_NAMES_hosuing,Total_Under_25,Ratio_Under_25
L_HOOD,,,,,,,,,,,,,,,,
Ballard,28,"Loyal Heights, Adams, Whittier Heights, West W...",1.046035e+08,59248.568772,"POLYGON ((-122.40266 47.69602, -122.40236 47.6...",WhittierHeightsBallardBallard,9270.0,7563.0,3567.0,1268.0,762.0,680.0,1271.0,"LoyalHeights,Adams,WhittierHeights,WestWoodlan...",4835.0,0.521575
Beacon Hill,29,"North Beacon Hill, Mid-Beacon Hill, South Beac...",1.479894e+08,78783.502732,"POLYGON ((-122.31281 47.59581, -122.31216 47.5...",NorthBeaconHill,1651.0,1027.0,354.0,237.0,164.0,40.0,232.0,"NorthBeaconHill,Mid-BeaconHill,SouthBeaconHill...",591.0,0.357965
Capitol Hill,30,"Portage Bay, Montlake, Madison Park, Broadway,...",1.068562e+08,75147.170565,"MULTIPOLYGON (((-122.29599 47.64184, -122.2960...",NorthCapitolHillMillerParkMadisonParkMadison-M...,6184.0,4227.0,2164.0,631.0,510.0,126.0,781.0,"PortageBay,Montlake,MadisonPark,Broadway,Steve...",2795.0,0.451973
Cascade,31,"Westlake, Eastlake, South Lake Union",2.717920e+07,48680.288698,"POLYGON ((-122.32124 47.65231, -122.32113 47.6...",EastlakeSouthLakeUnion,1323.0,853.0,472.0,144.0,67.0,64.0,106.0,"Westlake,Eastlake,SouthLakeUnionWestlake,Eastl...",616.0,0.465608
Central Area,32,"Madrona, Harrison/Denny-Blaine, Minor, Leschi,...",9.527258e+07,52688.475894,"POLYGON ((-122.27911 47.62786, -122.27915 47.6...",JudkinsPark,508.0,271.0,113.0,44.0,48.0,25.0,41.0,"Madrona,Harrison/Denny-Blaine,Minor,Leschi,Man...",157.0,0.309055
Delridge,33,"Highland Park, North Delridge, Riverview, High...",1.623506e+08,62979.283952,"POLYGON ((-122.35272 47.56439, -122.35253 47.5...",NorthDelridgeHighPointRiverviewHighlandParkHig...,9088.0,7053.0,3418.0,1132.0,823.0,508.0,1163.0,"HighlandPark,NorthDelridge,Riverview,HighPoint...",4550.0,0.500660
Downtown,34,"Pike-Market, Belltown, International District,...",5.537664e+07,51622.719582,"POLYGON ((-122.32929 47.5924, -122.32969 47.59...",FirstHillBelltown,3728.0,2530.0,1347.0,541.0,301.0,105.0,236.0,"Pike-Market,Belltown,InternationalDistrict,Cen...",1888.0,0.506438
Interbay,35,Interbay,3.350394e+07,64207.586761,"POLYGON ((-122.39728 47.66505, -122.39731 47.6...",Interbay,2116.0,1521.0,632.0,247.0,260.0,146.0,197.0,Interbay,879.0,0.415406
Lake City,36,"Victory Heights, Matthews Beach, Meadowbrook, ...",1.158068e+08,54114.407337,"POLYGON ((-122.28403 47.73358, -122.28403 47.7...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
test.to_file('../data/processed/q4Map_in.geojson', index=True)